# Level 3: Agentic RAG

This tutorial presents an example of executing queries with agentic RAG in Llama Stack. It shows how to initialize an agent with the RAG tool provided by Llama Stack and to invoke it such that retrieval from a vector DB is activated when necessary. The tutorial also covers document ingestion using the RAG tool.
For a foundational (non-agentic) RAG tutorial, please refer to [Level1_foundational_RAG.ipynb](demos/rag_agentic/notebooks/Level1_foundational_RAG.ipynb).

## Overview

This tutorial covers the following steps:
1. Connecting to a llama-stack server.
2. Indexing a collection of documents in a vector DB for later retrieval.
3. Initializing the agent capable of retrieving content from vector DB via tool use.
4. Launching the agent and using it to answer user queries during the inference step.


## Prerequisites

Before starting, ensure you have a running instance of the Llama Stack server (local or remote) with at least one preconfigured vector DB. For more information, please refer to the corresponding [Llama Stack tutorials](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

## 1. Setting Up the Environment
- Import the necessary libraries.
- Define the settings for the RAG pipeline, including the Llama Stack server URL, inference and document ingestion parameters.
- Initialize the connection to the server.

In [1]:
import os
import uuid

from llama_stack_client import Agent, AgentEventLogger, RAGDocument, LlamaStackClient

import sys
sys.path.append('..')  
from src.utils import step_printer

# the server endpoint
LLAMA_STACK_SERVER_URL = "http://localhost:8321"

# inference settings
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"
SYSTEM_PROMPT = "You are a helpful assistant. "
TEMPERATURE = 0.0
TOP_P = 0.95
MAX_TOKENS = 4096

# RAG settings
VECTOR_DB_EMBEDDING_MODEL = "all-MiniLM-L6-v2"
VECTOR_DB_EMBEDDING_DIMENSION = 384
VECTOR_DB_CHUNK_SIZE = 512

# For this demo, we are using Milvus Lite, which is our preferred solution. Any other Vector DB supported by Llama Stack can be used.
VECTOR_DB_PROVIDER_ID = 'milvus'

# initialize the inference strategy
if TEMPERATURE > 0.0:
    strategy = {"type": "top_p", "temperature": TEMPERATURE, "top_p": TOP_P}
else:
    strategy = {"type": "greedy"}

# set this setting to True to stream the agent reply
STREAM_OUTPUT = False
    
# initialize the document collection to be used for RAG
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
    
# initialize the server connection
client = LlamaStackClient(base_url=os.environ.get("LLAMA_STACK_ENDPOINT", LLAMA_STACK_SERVER_URL))

## 2. Indexing the Documents
- Initialize a new document collection in the target vector DB. All parameters related to the vector DB, such as the embedding model and dimension, must be specified here.
- Provide a list of document URLs to the RAG tool. Llama Stack will handle fetching, conversion and chunking of the documents' content.

In [3]:
# define and register the document collection to be used
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=VECTOR_DB_EMBEDDING_MODEL,
    embedding_dimension=VECTOR_DB_EMBEDDING_DIMENSION,
    provider_id=VECTOR_DB_PROVIDER_ID,
)

# ingest the documents into the newly created document collection
urls = [
    ("https://www.openshift.guide/openshift-guide-screen.pdf", "application/pdf"),
    ("https://www.cdflaborlaw.com/_images/content/2023_OCBJ_GC_Awards_Article.pdf", "application/pdf"),
]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=url,
        mime_type=url_type,
        metadata={},
    )
    for i, (url, url_type) in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=VECTOR_DB_CHUNK_SIZE,
)

## 3. Executing queries via the RAG-aware agent
- Initialize an agent with a list of tools including the built-in RAG tool. The RAG tool specification must include a list of document collection IDs to retrieve from.
- For each prompt, initialize a new agent session, execute a turn during which a retrieval call may be requested, and output the reply received from the agent.

In [4]:
queries = [
    "How to install OpenShift?",
    "Are employees based in California eligible for remote work?",
]

# initializing the agent
agent = Agent(
    client,
    model=MODEL_ID,
    instructions=SYSTEM_PROMPT,
    sampling_params={
        "strategy": strategy,
        "max_tokens": MAX_TOKENS,
    },
    # we make our agent aware of the RAG tool by including builtin::rag/knowledge_search in the list of tools
    tools=[
        dict(
            name="builtin::rag/knowledge_search",
            args={
                "vector_db_ids": [vector_db_id],  # list of IDs of document collections to consider during retrieval
            },
        )
    ],
)

for prompt in queries:
    print(f"User> {prompt}")
    
    # create a new turn with a new session ID for each prompt
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=agent.create_session(f"rag-session_{uuid.uuid4()}"),
        stream=STREAM_OUTPUT,
    )
    
    # print the response, including tool calls output
    if STREAM_OUTPUT:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)

User> How to install OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: knowledge_search, Arguments: {'query': 'installing OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 1:\nDocument_id:num-0\nContent:  We\nrecommend you to check the official Red Hat OpenShift Local documentation for an updated list of\nrequirements at the official documentation website.\n\uf05a\nRegarding Linux, even if Red Hat does not officially support them, OpenShift Local\ncan run on other distributions, such as Ubuntu or Debian, with minor caveats.\nRunning OpenShift Local on any Linux distribution requires a few additional\nsoftware packages to be installed through your default package manager. The\n15\ndocumentation at crc.dev/crc has more information about this subject.\n7.2. Hardware Requirements\nIn terms of hardware, OpenShift Local has some strict requirements. Your system must use a recent\nIntel CPU (except for Macs, where Apple Silicon machines are supported) with at least four physical\ncores and have at least 16 GB of RAM. Be aware that the base installation of OpenShift Local\nrequires at least 9 GB free to start. Of course, to run other applications on OpenShift Local, you will\nneed more RAM, so using a computer with at least 32 GB of RAM is strongly recommended.\nOpenShift Local also requires at least 35 GB of free disk space for its installation. The memory\nrequirements are likely to increase in the future, so please check the documentation at crc.dev for\nmore up-to-date information.\n7.3. Installation\nTo install OpenShift Local, open your web browser and navigate to console.redhat.com/openshift/\ncreate/local . Download the latest release of OpenShift Local and the "pull secret" file. The latter is a\nfile containing a key identifying your copy of OpenShift Local to your Red Hat Developer account.\nUnzip the file containing the OpenShift Local executable, and using your terminal, run the\ncommand crc setup . This command will prepare your copy of OpenShift Local, verifying\nrequirements and setting the required configuration values.\nOnce the crc setup command is ready, launch crc start. Running crc start can take a long time,\naround 20 minutes, on a recent PC.\nOnce started, access the OpenShift Web Console with the crc console command, which will open\nyour default browser. OpenShift Local uses the developer username and password to log in as a\nlow-privilege user, while the kubeadmin user uses a random-generated password. Use the crc\nconsole --credentials command to find the credentials required to log in as the kubeadmin user.\nOpenShift Local allows developers to perform various everyday tasks as if it were a standard\nOpenShift cluster, like deploying applications\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 2:\nDocument_id:num-0\nContent: .\nThese characteristics set OpenShift apart as an excellent Kubernetes platform for enterprise users.\nThe latest version of OpenShift available at the time of this writing is 4.12.\n3.2. Is Red Hat OpenShift Open Source?\nRed Hat OpenShift is a commercial product based on an open-source project called OKD. This\nacronym means " OpenShift Kubernetes Distribution" and is publicly available for everyone to\ninspect and contribute. Like the upstream Kubernetes project, OKD developers use the Go\nprogramming language.\n3.3. How can I run OpenShift?\nToday, Red Hat OpenShift is available through various mechanisms and formats:\n• DevOps teams can install it in their data centers "on-premise."\n• Major hyperscalers such as AWS, Azure, Google Cloud Platform, and IBM Cloud offer managed\nRed Hat OpenShift installations.\n• Developers can either run OpenShift locally on their workstations using Red Hat OpenShift\nLocal, also known as CRC or "Code-Ready Containers"\n• They can also request a 30-day trial OpenShift cluster, offered by Red Hat, at no charge, for\ntesting and evaluation purposes.\nRed Hat OpenShift is an integrated Platform-as-a-Service f


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The provided text appears to be an excerpt from a book or documentation about Red Hat OpenShift, a platform for building, deploying, and managing applications in a cloud-native environment. The text covers various topics related to working with OpenShift, including:

1. **Importing Applications**: The text explains how to import applications into OpenShift using different methods:
	* Importing from a Git repository (e.g., GitHub, GitLab, Gitea).
	* Importing YAML files directly.
	* Importing JAR files for Java applications.
	* Creating a container image and importing it into OpenShift.
2. **Creating and Debugging Applications with the odo Tool**: The text introduces the odo tool, which is designed for software developers to create applications using "Devfiles" (devfile.yaml). Devfiles contain information about an application's programming language, dependencies, and other essential details. The odo tool can be downloaded 

[
│   TextContentItem(
│   │   text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 1:\nDocument_id:num-1\nContent:  Set clear performance expectations and key performance indicators (KPIs) \n     for remote employees.\n      • Implement regular performance reviews and feedback sessions, even if they\n      are conducted virtually.\n      • Use performance management software and tools to track and measure \n      remote employees’ contributions and achievements.\nConclusion\nBy addressing these five issues and implementing the associated compliance\nstrategies, California employers can successfully manage their remote workforce,\nmitigate legal risks, and create a legally-compliant remote work environment that\nbenefits both the employer and the remote workers. Continually adapting and\nrefining remote work policies and practices is the key to staying in compliance with\nCalifornia’s labor and employment laws and to meeting the evolving needs of the\nworkforce.\nFor questions regarding complying with California employment-related remote\nwork laws, please contact either author - Alessandra Whipple at\nawhipple@cdflaborlaw.com or Todd Wulffson at twulffson@cdflaborlaw.com. \nBy Alessandra C. Whipple, Esq. and Todd R. Wulffson, Esq.\nWhipple Wulffson\n\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 2:\nDocument_id:num-1\nContent: B-38 ORANGE COUNTY BUSINESS JOURNAL                                                                                              GENERAL COUNSEL AWARDS OCTOBER 9, 2023\nDuring and since the pandemic, the traditional office commute rapidly evolved\nfrom the familiar dystopia of So. California freeways, to the more comfortable\nmorning sweatpants shuffle down the hall to a desk or couch. As remote work\nappears to be lingering, and for some, may be here to stay, employers –\nespecially in California (infamous for its complex and ever-changing labor and\nemployment laws), face unique challenges and concerns. In this article, we will\nexplore the top five issues facing California employers\nregarding remote work and offer strategies for\ncompliance.\n1. Wage and Hour Compliance\nOne of the foremost concerns for California employers\nwith all employees, but especially with remote workers,\nis ensuring compliance with wage and hour laws.\nCalifornia has a complex web of regulations governing\novertime, meal and rest breaks, and record-keeping. If\nan employee performs services in California (remote\nworkers in other states cannot take advantage of\nCalifornia law), employers must track the hours worked\nby remote employees accurately, regardless of their location, and pay them in\naccordance with California’s labor laws.\nCompliance Strategies\n      • Implement time-tracking software to monitor remote employees’ work hours.\n      • Train managers and employees on timekeeping best practices and the \n      importance of accurate record-keeping.\n      • Develop clear remote work policies (preferably that remote workers sign), \n      that outline expectations regarding work hours and reporting time worked.\n• These policies should clearly state that all off-the-clock work is strictly \nprohibited. \n• Have employees confirm in writing that they have accurately reported all \nof their time worked.\n• Prohibit texting, e-mailing, and phone calls when employees are not \nclocked-in and educate and restrict managers from contacting hourly \nemployees after hours.\n      • Develop and enforce meal and rest break policies.\n• Consider developing a break schedule that is circulated to managers and \nremote workers, which will mitigate potential unintended meal   \ninterruptions. \n• Meal and rest break policies should clearly state that employees are \nprohibited from performing work while on break.\n• Employees should inform remote employees to report missed meal or rest \nbreaks, and


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the provided text, it appears that there are two separate topics being discussed: one related to navigating remote work challenges for California employers and the other related to Tekton and OpenShift.

For the first topic, the top five issues for California employers when it comes to remote work are:

1. Expense Reimbursement
2. Data Security and Privacy
3. Meal and Rest Breaks
4. Expense Reimbursement (again, with a focus on establishing a comprehensive policy)
5. Employee Classification

For the second topic, the text discusses Tekton and OpenShift, specifically how to create a CI/CD pipeline using YAML files and how to inspect the pipeline's status and logs.

If you're looking for information on how to navigate remote work challenges for California employers, I can provide some general guidance:

1. Establish a comprehensive expense reimbursement policy that outlines eligible expenses and the reimbursement p

## Key Takeaways
This tutorial demonstrates how to implement agentic RAG with Llama Stack. We do so by initializing an agent while giving it access to the RAG tool, then invoking the agent on each of the specified queries. Please check out our [complementary tutorial](demos/rag_agentic/notebooks/Level1_foundational_RAG.ipynb) for a non-agentic RAG example.